In [43]:
from copy import deepcopy
from itertools import product
import logging
import random

import pandas as pd
import numpy as np
from tabulate import tabulate
from termcolor import colored
from tqdm import tqdm


from word_grid import WordGrid, Direction



MIN_WORD_LEN = 3

In [44]:

def get_logger():
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.ERROR)
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.ERROR)
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    return logger

logger = get_logger()

In [45]:
puzzle = WordGrid((11,17))
puzzle

-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -

In [46]:
word_index = pd.read_csv("data/word_index.csv", encoding='utf-8')
dictionary = word_index[word_index["lang_code"] == "en"]
dictionary["word"] = dictionary["word"].astype(str)
dictionary = dictionary[dictionary["len"] >= MIN_WORD_LEN]
dictionary = dictionary[dictionary["len"] <= max(puzzle.shape)]
dictionary = dictionary[~dictionary["word"].str.contains(r"[0-9-]")]
print(len(dictionary))

C:\Users\charl\AppData\Local\Temp\ipykernel_3980\4029767802.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dictionary["word"] = dictionary["word"].astype(str)


1169246


In [47]:
puzzle.reset()
while not puzzle.add_word((1,2), Direction.ACROSS, dictionary["word"].sample(1).item()):
    pass
print(puzzle)

-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  t  a  r  t  a  r  i  z  a  t  i  o  n  s  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -


In [48]:
def get_candidates(puzzle: WordGrid, position: tuple, direction: Direction, blacklist: list):

    candidates = dictionary[dictionary["word"].apply(lambda w: puzzle.validate_word(position, direction, w))]
    candidates = candidates[~candidates["word"].isin(blacklist)]
        
    return candidates
    

In [49]:
puzzle.reset()

n = 0
seed = 50
target_n = 40
random.seed(seed)
direction = Direction.DOWN
word_list = []
snapshots = []
positions = {
    Direction.DOWN: {pos: [] for pos in product(range(puzzle.shape[1]), range(puzzle.shape[0] - MIN_WORD_LEN + 1))},
    Direction.ACROSS: {pos: [] for pos in product(range(puzzle.shape[1] - MIN_WORD_LEN + 1), range(puzzle.shape[0]))}
}
pbar = tqdm(total=target_n)
while n < target_n:
    if len(positions[direction]) == 0:
        if len(positions[Direction.flip(direction)]) == 0:
            break
        direction = Direction.flip(direction)
    
    position = random.choice(list(positions[direction]))
    
    blacklist = positions[direction][position] + word_list
    candidates = get_candidates(puzzle, position, direction, blacklist)

    if len(candidates) == 0:
        positions[direction].pop(position, None)
        continue
    
    try:
        word = candidates["word"].sample(1, weights=candidates.freq, random_state=seed).item()
    except:
        word = candidates["word"].sample(1, random_state=12).item()
    
    pbar.update(1)
    pbar.set_description(f"word: {word}, pos: {position}, dir: {direction.name.lower()}, cnd: {len(candidates)}, slots {len(positions[Direction.DOWN])}d {len(positions[Direction.ACROSS])}a", refresh=True)
    
    if puzzle.add_word(position, direction, word):
        snapshots.append(({"position": position, "direction": direction, "word": word}, deepcopy(puzzle)))
        if len(positions[Direction.flip(direction)]) > 0:
            direction = Direction.flip(direction)

        positions[direction].pop(position, None)    
        word_list.append(word)
        n += 1
    else:
        positions[direction][position].append(word)
        logger.info(f"Can't place word {word} at {position}")

print(word_list)
print(puzzle)

word: in place, pos: (9, 0), dir: down, cnd: 372, slots 147d 161a:  22%|██▎       | 9/40 [02:37<09:03, 17.53s/it]


['down', 'they', 'which', 'time', 'for', 'when', 'were', 'known', 'still', 'cost', 'that', 'had', 'one', 'port', 'with', 'its', 'each', 'shot', 'new', 'against', 'other', 'John', 'days', 'fell', 'Sejm', 'over', 'more', 'her', 'can', 'mhw', 'run', 'the', 'ash', 'des', 'Eve', 'add', 'and', 'lamp', 'not', 'eds']
-  -  s  h  o  t  -  h  e  r  -  t  h  e  -  e  -
o  n  -  -  -  f  e  l  l  -  -  -  -  a  d  d  -
t  o  -  -  -  -  t  h  e  y  -  -  -  c  o  s  t
h  t  t  d  -  p  o  r  t  -  -  -  m  h  w  -  -
e  -  h  a  d  -  -  k  n  o  w  n  -  -  n  w  -
r  -  a  y  -  w  s  -  -  n  h  -  -  -  -  e  m
-  i  t  s  -  i  t  -  s  e  i  r  -  f  e  r  o
a  -  -  -  -  t  i  m  e  -  c  u  -  o  v  e  r
n  n  a  -  c  h  l  -  j  o  h  n  -  r  e  -  e
d  e  s  -  a  -  l  a  m  p  -  -  -  -  -  -  -
-  w  h  e  n  -  -  -  -  -  a  g  a  i  n  s  t


In [54]:
for params, grid in snapshots:
    position, direction, word = list(params.values())
    print(f"word: {word}, pos: {position}, dir: {direction.name.lower()}")
    print(grid)

word: down, pos: (14, 1), dir: down
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  d  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  o  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  w  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  n  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
word: they, pos: (6, 2), dir: across
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  d  -  -
-  -  -  -  -  -  t  h  e  y  -  -  -  -  o  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  w  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  n  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  

In [51]:
def custom_print(puzzle: WordGrid):
    to_print = []
    for chars, states in zip(puzzle.puzzle, puzzle.state):
        data = []
        for char, state in zip(chars, states):
            if state & Direction.ACROSS.value and state & Direction.DOWN.value:
                color = "magenta"
            elif state & Direction.ACROSS.value:
                color = "blue"
            elif state & Direction.DOWN.value:
                color = "yellow"
            else:
                color = "white"
            data.append(colored(char, color))
        to_print.append(data)
    print(tabulate(to_print))

In [52]:
custom_print(puzzle)

-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
-  -  s  h  o  t  -  h  e  r  -  t  h  e  -  e  -
o  n  -  -  -  f  e  l  l  -  -  -  -  a  d  d  -
t  o  -  -  -  -  t  h  e  y  -  -  -  c  o  s  t
h  t  t  d  -  p  o  r  t  -  -  -  m  h  w  -  -
e  -  h  a  d  -  -  k  n  o  w  n  -  -  n  w  -
r  -  a  y  -  w  s  -  -  n  h  -  -  -  -  e  m
-  i  t  s  -  i  t  -  s  e  i  r  -  f  e  r  o
a  -  -  -  -  t  i  m  e  -  c  u  -  o  v  e  r
n  n  a  -  c  h  l  -  j  o  h  n  -  r  e  -  e
d  e  s  -  a  -  l  a  m  p  -  -  -  -  -  -  -
-  w  h  e  n  -  -  -  -  -  a  g  a  i  n  s  t
-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -


In [53]:
puzzle.validate_word((5, 2), Direction.DOWN, "mad")

False